# Test smple_weight, kernels, C, sklearn estimator

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from stree import Stree
import time

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state)
    return Xtrain, Xtest, ytrain, ytest

data = load_creditcard(-5000) # Take all true samples with up to 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
# data = load_creditcard(-1000) # Take 1000 samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
_, data = np.unique(ytrain, return_counts=True)
wtrain = (data[1] / np.sum(data),  data[0] / np.sum(data))
_, data = np.unique(ytest, return_counts=True)
wtest = (data[1] / np.sum(data),  data[0] / np.sum(data))
# Set weights inverse to its count class in dataset
weights = np.ones(Xtrain.shape[0],)
weights[ytrain==0] = wtrain[0]
weights[ytrain==1] = wtrain[1]
weights_test = np.ones(Xtest.shape[0],)
weights_test[ytest==0] = wtest[0]
weights_test[ytest==1] = wtest[1]
print(weights[:4], weights_test[:4])

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (5492, 28)  y.shape (5492,)
Fraud: 9.068% 498
Valid: 90.932% 4994
[0.09131113 0.09131113 0.09131113 0.09131113] [0.08919903 0.08919903 0.08919903 0.08919903]


# Tests

## Test simple_weights
Compute accuracy with weights in samples. The weights are set based on the inverse of the number of samples of each class

In [5]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

Accuracy of Train without weights 0.9901144640998959
Accuracy of Train with    weights 0.9924557752341311
Accuracy of Tests without weights 0.9872572815533981
Accuracy of Tests with    weights 0.9824029126213593


## Test accuracy with different kernels
Compute accuracy on train and test set with default hyperparmeters of every kernel

In [6]:
random_state=1
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = Stree(C=7, kernel=kernel, random_state=random_state).fit(Xtrain, ytrain)
    accuracy_train = clf.score(Xtrain, ytrain)
    accuracy_test = clf.score(Xtest, ytest)
    time_spent = time.time() - now
    print(f"Time: {time_spent:.2f}s\tKernel: {kernel}\tAccuracy_train: {accuracy_train}\tAccuracy_test: {accuracy_test}")


Time: 10.77s	Kernel: linear	Accuracy_train: 0.986732570239334	Accuracy_test: 0.9860436893203883
Time: 1.56s	Kernel: rbf	Accuracy_train: 0.9945369406867846	Accuracy_test: 0.9896844660194175
Time: 1.23s	Kernel: poly	Accuracy_train: 0.9942767950052029	Accuracy_test: 0.9860436893203883


## Test diferent values of C

In [ ]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, kernel="linear", random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

## Test iterator
Check different weays of using the iterator

In [9]:
#check iterator
for i in list(clf):
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.1659
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0139
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([270]))
root - Down - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.2311
root - Down - Up - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([13]))
root - Down - Up - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([2]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0375
root - Up - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 

In [10]:
#check iterator again
for i in clf:
    print(i)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.1659
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0139
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([270]))
root - Down - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.2311
root - Down - Up - Down, <pure> - Leaf class=1 belief= 1.000000 impurity=0.0000 counts=(array([1]), array([13]))
root - Down - Up - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([2]))
root - Up feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.0375
root - Up - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 

## Test STree is a sklearn estimator

In [11]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

1 functools.partial(<function check_no_attributes_set_in_init at 0x11e6f3f80>, 'Stree')
2 functools.partial(<function check_estimators_dtypes at 0x11e6ef0e0>, 'Stree')
3 functools.partial(<function check_fit_score_takes_y at 0x11e6e9f80>, 'Stree')
4 functools.partial(<function check_sample_weights_pandas_series at 0x11e6e68c0>, 'Stree')
5 functools.partial(<function check_sample_weights_not_an_array at 0x11e6e69e0>, 'Stree')
6 functools.partial(<function check_sample_weights_list at 0x11e6e6b00>, 'Stree')
7 functools.partial(<function check_sample_weights_shape at 0x11e6e6c20>, 'Stree')
8 functools.partial(<function check_sample_weights_invariance at 0x11e6e6d40>, 'Stree')
9 functools.partial(<function check_estimators_fit_returns_self at 0x11e6f30e0>, 'Stree')
10 functools.partial(<function check_estimators_fit_returns_self at 0x11e6f30e0>, 'Stree', readonly_memmap=True)
11 functools.partial(<function check_complex_data at 0x11e6e6ef0>, 'Stree')
12 functools.partial(<function check_dt

In [12]:
# Check if the classifier is a sklearn estimator
check_estimator(Stree())

## Compare to SVM

In [13]:
svc = SVC(C=7, kernel='rbf', gamma=.001, random_state=random_state)
clf = Stree(C=17, kernel='rbf', gamma=.001, random_state=random_state)
svc.fit(Xtrain, ytrain)
clf.fit(Xtrain, ytrain)
print("== Not Weighted ===")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
svc.fit(Xtrain, ytrain, weights)
clf.fit(Xtrain, ytrain, weights)
print("==== Weighted =====")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
print("*SVC test score ..:", svc.score(Xtest, ytest, weights_test))
print("*STree test score :", clf.score(Xtest, ytest, weights_test))

== Not Weighted ===
SVC train score ..: 0.9820499479708636
STree train score : 0.9830905306971904
SVC test score ...: 0.9842233009708737
STree test score .: 0.9848300970873787
==== Weighted =====
SVC train score ..: 0.9768470343392299
STree train score : 0.9758064516129032
SVC test score ...: 0.975121359223301
STree test score .: 0.9781553398058253
*SVC test score ..: 0.9433869483836175
*STree test score : 0.9542572525345916


In [14]:
print(clf)

root feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.1659
root - Down feaures=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27) impurity=0.2701
root - Down - Down, <cgaf> - Leaf class=1 belief= 0.841270 impurity=0.2671 counts=(array([0, 1]), array([ 60, 318]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 impurity=0.0000 counts=(array([0]), array([1]))
root - Up, <cgaf> - Leaf class=0 belief= 0.990476 impurity=0.0189 counts=(array([0, 1]), array([3432,   33]))



## Test max_features

In [15]:
for max_features in [None, "auto", "log2", 7, .5, .1, .7]:
    now = time.time()
    print("*"*40)
    clf = Stree(random_state=random_state, max_features=max_features)
    clf.fit(Xtrain, ytrain)
    print(f"max_features {max_features} = {clf.max_features_}")
    print("Train score :", clf.score(Xtrain, ytrain))
    print("Test score .:", clf.score(Xtest, ytest))
    print(f"Took {time.time() - now:.2f} seconds")

****************************************
max_features None = 28
Train score : 0.9846514047866806
Test score .: 0.9860436893203883
Took 5.03 seconds
****************************************
max_features auto = 5
Train score : 0.9802289281997919
Test score .: 0.9824029126213593
Took 2.20 seconds
****************************************
max_features log2 = 4
Train score : 0.9765868886576483
Test score .: 0.9769417475728155
Took 0.91 seconds
****************************************
max_features 7 = 7
Train score : 0.9768470343392299
Test score .: 0.9781553398058253
Took 4.55 seconds
****************************************


KeyboardInterrupt: 

In [23]:
from itertools import combinations
max_features=3
features = range(Xtrain.shape[1])
features_sets = list(combinations(features, max_features))
print(features_sets)

(6, 10, 14), (6, 10, 15), (6, 10, 16), (6, 10, 17), (6, 10, 18), (6, 10, 19), (6, 10, 20), (6, 10, 21), (6, 10, 22), (6, 10, 23), (6, 10, 24), (6, 10, 25), (6, 10, 26), (6, 10, 27), (6, 11, 12), (6, 11, 13), (6, 11, 14), (6, 11, 15), (6, 11, 16), (6, 11, 17), (6, 11, 18), (6, 11, 19), (6, 11, 20), (6, 11, 21), (6, 11, 22), (6, 11, 23), (6, 11, 24), (6, 11, 25), (6, 11, 26), (6, 11, 27), (6, 12, 13), (6, 12, 14), (6, 12, 15), (6, 12, 16), (6, 12, 17), (6, 12, 18), (6, 12, 19), (6, 12, 20), (6, 12, 21), (6, 12, 22), (6, 12, 23), (6, 12, 24), (6, 12, 25), (6, 12, 26), (6, 12, 27), (6, 13, 14), (6, 13, 15), (6, 13, 16), (6, 13, 17), (6, 13, 18), (6, 13, 19), (6, 13, 20), (6, 13, 21), (6, 13, 22), (6, 13, 23), (6, 13, 24), (6, 13, 25), (6, 13, 26), (6, 13, 27), (6, 14, 15), (6, 14, 16), (6, 14, 17), (6, 14, 18), (6, 14, 19), (6, 14, 20), (6, 14, 21), (6, 14, 22), (6, 14, 23), (6, 14, 24), (6, 14, 25), (6, 14, 26), (6, 14, 27), (6, 15, 16), (6, 15, 17), (6, 15, 18), (6, 15, 19), (6, 15, 20),

In [25]:
import random
if len(features_sets) > 3:
    features_sets = random.sample(features_sets, 3)
print(features_sets)

[(3, 7, 10), (1, 7, 24), (7, 12, 26)]
